# Pure Python Pipeline
Ripple noise removal, motion correction, trace deconvoloution and extraction in one notebook. This should replace demo_pipeline as the standard analysis notebook! 

## TODO list (from demo_pipeline)
- TODO: save shifts in CNMF.estimates.shifts! Or is it too large?
- TODO: manually reject components?
- TODO: rename temporal output memmap files as checkpoint_x (1, 2, ...)?
- TODO: hdf5_to_numpy as reverse of numpy_to_hdf5 (RNR export step)
- TODO: check demo_motion_correction.ipynb for evaluation of motion correction
- TODO: appropriate file variable names (fname_new ... are NOT appropriate).
- TODO: save file names should be more meaningful, for easier cleanup.
- TODO: rnr save file: should have rnr in the hdf5 file name, otherwise too confusing!
- TODO: create a small txt file with the file names and their purpose (whether they can be deleted, how to use them, what do they contain) at some point (early) in the analysis, save it in export folder.
- TODO: cnm.estimates.idx_components and idx_components_bad are removed at one step. Instead, there is cnm.estimates.discarded_components. Do we need manual accept/reject?
- FIXME: _pars.json and _results.hdf5 contain nd2 file name twice: T301_tmev_d1T301_tmev_d1.270820.1110_22-10-20_14-18-40_pars.json and T301_tmev_d1T301_tmev_d1.270820.1110_22-10-20_14-18-40_results.hdf5
- IMPORTANT: make it more convenient to enter pipeline from any point. This includes defining parameters in one location, naming variables appropriately (F memmap, C memmap, nd2 file, hd5 file...) so user is aware which file they are supposed to open at which point of entry into analysis!
- memmap_ results in conflicting names if recordings are from same day. Include date time of analysis in name? Not a big problem as it is temporary file
- Study parallel processing of caiman (start server step, cleaning up server). It might be useful for RNR too.
- Evaluating components: cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview), the contents of model/ in CaImAn are used but looking for the model files in another directory (in my case, Users/Bence/caiman_data/model/)
- Export h5 file should have date time in filename to avoid overwriting. Both raw data and final results!
- Plot with slider: watch all the frames, compare RNR and original, then MC and original/RNR... QC
- Save RNR directly to memmap (opening as caiman movie, save to memmap?)? Although the problem is how slow RNR is...
- Maybe working with numpy array in motion correction (movie.motion_correct) is not that bad? Although no parameters...
- Plot frame before RNR and after RNR to set parameters... Interactive?
- Read Tips on analysis: https://caiman.readthedocs.io/en/master/CaImAn_Tips.html#motion-correction-tips
- RNR results in 4x size (uint16 to float64)! Need to clean up or use uint16 again.
- Check 2-channel recordings. Might want to save red channel, too, for matching?
- Save memmap files is inconsistent in naming (C order is memmap__d1_512_d2_512_d3_1_order_C_frames_577_ instead of T386_20211202_green_ex_els__d1_512_d2_512_d3_1_order_C_frames_577_)
- Include nd2 to h5 here (from nd2 to multipage tiff test.ipynb)
- It takes a lot of time to open nd2 file. Useful to copy data to be analyzed to local HDD on a previous day?
- way to manually reject/accept components
- IMPORTANT: https://caiman.readthedocs.io/en/master/On_file_types_and_sizes.html caiman works best when files are 1-2 GB big! It means we might want to split them in small pieces, or make sure they are multi-page tiff files!

## Import packages

In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
from RippleNoiseRemoval import RNR
import labrotation.file_handling as fh
import h5py
from time import time

import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from labrotation import file_handling as fh
from copy import deepcopy
try:
    cv2.setNumThreads(0)
except():
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour

import json  # for exporting parameters

# for exporting moco data:
from caiman.motion_correction import sliding_window
import cv2

import pandas as pd  # for opening data documentation
import warnings
import uuid  # for generating UUID in case of missing value

from movie_splitting import numpy_to_hdf5
bpl.output_notebook()

In [ ]:
log_fname = fh.choose_dir_for_saving_file("Select folder to save log file", fh.get_filename_with_date("caim_log", ".txt"))
print(f"Saving log file to\n{log_fname}")


## Set up logging (optional)

In [ ]:
logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",\
                    filename=log_fname,
                    level=logging.WARNING)

## Set input and output files

In [ ]:
# nd2 input file
nd2_fpath = fh.open_file("Select nd2 file")

# set folder to export temporary and result files
export_folder = fh.open_dir("Select folder to save results", True)

In [ ]:
nd2_fname = os.path.split(nd2_fpath)[-1]
# export_fname: get rid of .nd2 extension, append date and .h5 extension
#export_fname = fh.get_filename_with_date(os.path.splitext(os.path.split(nd2_fpath)[1])[0] + "_caim", ".h5")
#export_hd5_fpath = os.path.join(export_folder, export_fname)
#print(f"Export file selected: {export_hd5_fpath}")

results_root = fh.get_filename_with_date(os.path.splitext(os.path.split(nd2_fpath)[1])[0], "")

rnr_fname = results_root + "_rnr.hdf5"
rnr_fpath = os.path.join(export_folder, rnr_fname)

# input for motion correction; moco comes after RNR
moco_fnames = [rnr_fpath]

# rnr_fpath should be hdf5 for now. Not sure if MoCo/CaImAn supports h5.
assert rnr_fpath.split(".")[-1] in ["h5", "hdf5"], f"Invalid file extension: .{rnr_fpath.split('.')[-1]}, expected .h5"

cnmf_results_save_path = os.path.join(export_folder, results_root + "_cnmf.hdf5")  # caiman only supports saving hdf5, not h5

json_fname = results_root + "_pars.json"
json_fpath = os.path.join(export_folder, json_fname)

moco_pars_fname = results_root + "_moco_pars.h5"
moco_pars_fpath = os.path.join(export_folder, moco_pars_fname)

denoised_optional_fpath = os.path.join(export_folder, results_root + "_denoised.tif") 

print(f"Input file selected:\n\t{nd2_fpath}")

print(f"Temporary file after RNR will be saved as\n\t{rnr_fpath}")
print(f"Going to perform MoCo on\n\t{moco_fnames}")
print(f"Results of trace extraction will be saved as\n\t{cnmf_results_save_path}")
print(f"Parameters will be saved as\n\t{json_fpath}")
print(f"MoCo parameters will be saved as\n\t{moco_pars_fpath}")
print(f"\nOptional denoised results will be saved as\n\t{denoised_optional_fpath}")

# Add UUID

In [ ]:
data_docu_folder = fh.open_dir("Open Data Documentation folder")

In [ ]:
docu_files_list = []
session_uuid = None
for root, dirs, files in os.walk(data_docu_folder):
    for name in files:
        if "grouping" in name:
            if "~" in name: # "~" on windows is used for temporary files that are opened in excel
                docu_files_list = []
                raise Exception(f"Please close all excel files and try again. Found temporary file in:\n{os.path.join(root, name)}")
            fpath = os.path.join(root, name)
            df = pd.read_excel(fpath)
            df = df[df["nd2"] == nd2_fname]
            if len(df) > 0:
                if len(df) > 1:
                    raise Exception(f"File name appears several times in data documentation:\n\t{nd2_fname}\n{df}")
                else:
                    session_uuid = df["uuid"].iloc[0]
                break
            docu_files_list.append(fpath)
if session_uuid is None:
    session_uuid = uuid.uuid4().hex 
    warnings.warn(f"Warning: movie does not have entry (uuid) in data documentation!\nYou should add data to documentation. The generated uuid for this session is: {session_uuid}", UserWarning)
print(f"UUID is {session_uuid}")

## Ripple Noise Removal

In [ ]:
win = 40
amplitude_threshold = 10.8

In [ ]:
rnr = RNR(win, amplitude_threshold) 

In [ ]:
#begin_end_frames = None  # (begin, end): if want to work with part of the file
t0_open = time()
if "begin_end_frames" in locals():
    rnr.open_recording(nd2_fpath, begin_end_frames)  # opens usual recording size (8.8-9 GB) in about 830 s
else:
    rnr.open_recording(nd2_fpath)
    begin_end_frames = (1, len(rnr.nd2_data))
print(f"File opened in {time() - t0_open} s")

In [ ]:
t0_single = time()
rnr_data = rnr.rnr_singlethread()  # a bit faster than opening file, around 500s for 8.8-9 GB
t1_single = time()
print(f"RNR single thread finished in {t1_single - t0_single} s")
print(f"Result is a {type(rnr_data)} with datatype {rnr_data.dtype}")
print(f"Shape: {rnr_data.shape[0]} frames of {rnr_data.shape[1]}x{rnr_data.shape[2]} pixels")

### Export RNR movie to hd5 file.
The reason to this otherwise unnecessary step is that motion correction cannot work from numpy array... Or at least the movie.motion_correct() does not have many options. See https://caiman.readthedocs.io/en/master/core_functions.html#movie-handling motion_correct

In [ ]:
numpy_to_hdf5(rnr_data, rnr_fpath)
#dataset_name = "mov"  # var_name_hdf5 in various functions refers to this name! Default is always mov.
#with h5py.File(export_hd5_fpath, 'w') as hf:
#    dataset = hf.create_dataset(dataset_name, shape=rnr_data.shape, dtype=np.float64)  # TODO: float64 is much larger file!
#    for i_frame in range(rnr_data.shape[0]):
#        dataset[i_frame] = rnr_data[i_frame]


## Motion Correction

### Optional: split up video into segments to process and skip
Important: the numbering should correspond to the nd2 indexing, i.e. from 1 to n_frames! No zero-indexing here!

In [ ]:
# set intervals for MoCo. It seems to be more resilient to unusual signal, so applying it for whole movie often makes sense
moco_intervals = [(1,102), (103, 428), (429, 482), (483, 577)]
moco_flags = [False, True, False, True]
cnmf_intervals = moco_intervals.copy()
cnmf_flags = [True, True, False, True]

assert len(cnmf_flags) == len(cnmf_intervals)
assert len(moco_flags) == len(moco_intervals)

### Optional: Play the movie

In [ ]:
display_movie = False
if display_movie:
    ds_ratio = 0.2
    movie.resize(1, 1, ds_ratio).play(
        q_max=99.5, fr=30, magnification=2)  # this should not change size of movie itself

### Setup some parameters
We set some parameters that are relevant to the file, and then parameters for motion correction, processing with CNMF and component quality evaluation. Note that the dataset `Sue_2x_3000_40_-46.tif` has been spatially downsampled by a factor of 2 and has a lower than usual spatial resolution (2um/pixel). As a result several parameters (`gSig, strides, max_shifts, rf, stride_cnmf`) have lower values (halved compared to a dataset with spatial resolution 1um/pixel).

### Create a parameters object
You can creating a parameters object by passing all the parameters as a single dictionary. Parameters not defined in the dictionary will assume their default values. The resulting `params` object is a collection of subdictionaries pertaining to the dataset to be analyzed `(params.data)`, motion correction `(params.motion)`, data pre-processing `(params.preprocess)`, initialization `(params.init)`, patch processing `(params.patch)`, spatial and temporal component `(params.spatial), (params.temporal)`, quality evaluation `(params.quality)` and online processing `(params.online)`

In [ ]:
# dataset dependent parameters
fr = 15                             # imaging rate in frames per second
decay_time = 0.4                    # length of a typical transient in seconds

# motion correction parameters
strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system
gnb = 2                     # number of global background components
merge_thr = 0.85            # merging threshold, max correlation allowed
rf = 15                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 6             # amount of overlap between the patches in pixels
K = 4                       # number of components per patch
gSig = [4, 4]               # expected half size of neurons in pixels
method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization

# parameters for component evaluation
min_SNR = 2.0               # signal to noise ratio for accepting a component
rval_thr = 0.85              # space correlation threshold for accepting a component
cnn_thr = 0.99              # threshold for CNN based classifier
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

In [ ]:
opts_dict = {'fnames': moco_fnames, 
            'fr': fr,
            'decay_time': decay_time,
            'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid,
            'p': p,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub,
            'merge_thr': merge_thr, 
            'min_SNR': min_SNR,
            'rval_thr': rval_thr,
            'use_cnn': True,
            'min_cnn_thr': cnn_thr,
            'cnn_lowest': cnn_lowest,
            'var_name_hdf5': 'data',
            'gSig' :  gSig,}  # FIXME: does not work! Check where does this setting get lost?

opts = params.CNMFParams(params_dict=opts_dict)

### Setup a cluster
To enable parallel processing a (local) cluster needs to be set up. This is done with a cell below. The variable `backend` determines the type of cluster used. The default value `'local'` uses the multiprocessing package. The `ipyparallel` option is also available. More information on these choices can be found [here](https://github.com/flatironinstitute/CaImAn/blob/master/CLUSTER.md). The resulting variable `dview` expresses the cluster option. If you use `dview=dview` in the downstream analysis then parallel processing will be used. If you use `dview=None` then no parallel processing will be employed.

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
mc = MotionCorrect(moco_fnames, dview=dview, **opts.get_group('motion'))

In [ ]:
# TODO: ALTERNATIVE to exporting h5 and importing it again!
"""
Args:
            max_shift_w,max_shift_h: maximum pixel shifts allowed when correcting
                                     in the width and height direction

            template: if a good template for frame by frame correlation exists
                      it can be passed. If None it is automatically computed

            method: depends on what is installed 'opencv' or 'skimage'. 'skimage'
                    is an order of magnitude slower

            num_frames_template: if only a subset of the movies needs to be loaded
                                 for efficiency/speed reasons
                                 
max_shift_w=5,
max_shift_h=5,
num_frames_template=None,
template=None,
method: str = 'opencv',
remove_blanks: bool = False,
interpolation: str = 'cubic'
"""

# movie.motion_correct()   # this might change movie itself! Alternative: extract_shifts, apply_shifts

### Perform motion correction and save as C-order memmap
The filename is mc.fname_tot_els

## Perform MoCo on whole movie first, do not save the results yet

In [ ]:
#%%capture
#%% Run piecewise-rigid motion correction using NoRMCorre
mc.motion_correct(save_movie=False)

## Drop calculated shift values for frames where MoCo should not be performed

In [ ]:
x_shifts_els = deepcopy(mc.x_shifts_els)
y_shifts_els = deepcopy(mc.y_shifts_els)

In [ ]:
x_shape = x_shifts_els[0].shape
y_shape = y_shifts_els[0].shape
for i_piece, frames_tuple in enumerate(moco_intervals):
    if not moco_flags[i_piece]:  # skip this piece = set shifts to zero
         for i_frame in range(frames_tuple[0] -1 , frames_tuple[1]):  # include last frame as well
            x_shifts_els[i_frame] = np.zeros(x_shape)
            y_shifts_els[i_frame] = np.zeros(y_shape)

In [ ]:
mc.x_shifts_els = x_shifts_els
mc.y_shifts_els = y_shifts_els

In [ ]:
work_folder = os.path.split(moco_fnames[0])[0]
print(f"Changing work folder to {work_folder}, this is where moco result will be saved")

In [ ]:
os.chdir(work_folder)
print(f"Changed folder.")

In [ ]:
# should save in C order because cm.load_memmap() takes C-memmap. However, as the opening and closing of memmap files is 
# so confusing, I decided to try to copy the original demo_pipeline jupyter notebook as closely as I can.
moco_mmap_fname = mc.apply_shifts_movie(moco_fnames, save_memmap=True,order="F")
print(moco_mmap_fname)

In [ ]:
moco_mmap_fpath = os.path.join(work_folder, moco_mmap_fname)

In [ ]:
m_els = cm.load(moco_mmap_fpath)
border_to_0 = 0 if mc.border_nan is 'copy' else mc.border_to_0  # FIXME: gives warning, should use "==" with literals
    # maximum shift to be used for trimming against NaNs

# Save tif file (Optional, for checking MoCo)

In [ ]:
# TODO: float64 causes 4x file size! (35-36 GB instead of 8-9 GB)
save_moco_tif = False
if save_moco_tif:
    moco_tif_fpath = os.path.join(export_folder, results_root + "_moco.tif")
    m_els.save(moco_tif_fpath)
    print(f"MoCo results saved to {moco_tif_fpath}")

### Optional: show comparison with original movie

In [ ]:
#%% compare with original movie
display_movie = False  # TODO: does not seem to work. Create own function to show result?
if display_movie:
    m_orig = cm.load_movie_chain(moco_fnames)
    ds_ratio = 0.2
    cm.concatenate([m_orig.resize(1, 1, ds_ratio) - mc.min_mov*mc.nonneg_movie,
                    m_els.resize(1, 1, ds_ratio)], 
                   axis=2).play(fr=60, gain=15, magnification=2, offset=0)  # press q to exit

## Save now as C memmap, apply border_to_0
`fname_mmap_c` is the split file that we can handle just the same as `fname_new` in the original (Pure Python Pipeline) notebook.

In [ ]:
# this is a very stupid step, basically save the results again (mc_mmap) but with border_to_0 param in addition
fname_mmap_c = cm.save_memmap([moco_mmap_fpath], base_name='memmap', order='C',border_to_0=border_to_0, dview=dview) # exclude borders

# Work with MoCo-exported C memmap

In [ ]:
if 'fname_mmap_c' not in locals():
    fname_mmap_c = fh.open_file("Select C-memmap file.")
print(f"Working with C-memmap\n{fname_mmap_c}")

In [ ]:
# now load the file
Yr, dims, T = cm.load_memmap(fname_mmap_c)
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    #load frames in python format (T x X x Y)

In [ ]:
resolution = images[0].shape
resolution

In [ ]:
images.shape

### Create ndarray for concatenated CNMF segments
i.e. parts of the video specified in cnmf_intervals. The other segments contain weird signal that hinder the moco and CNMF.

In [ ]:
total_len = 0
len_concat = 0  # length of concatenated moco-segments video
for i_segment, segment_tuple in enumerate(cnmf_intervals):
    len_segment = segment_tuple[1] - segment_tuple[0] + 1
    if cnmf_flags[i_segment]:  # if moco was applied, add segment
        len_concat += len_segment
        
    total_len += len_segment
        
len_concat

In [ ]:
moco_split_memmap_fname = cm.paths.memmap_frames_filename("moco_split", resolution, len_concat, "C")
moco_split_memmap_fpath = os.path.join(work_folder, moco_split_memmap_fname)
print(f"Split-up moco C memmap for CNMF will be saved as\n\t{moco_split_memmap_fpath}")

In [ ]:
Yr.shape

In [ ]:
data_concatenated = np.zeros(shape=(len_concat, resolution[0], resolution[1]), dtype=images.dtype)
i_concat = 0  # index of frame in concatenated data
for i_segment, segment in enumerate(cnmf_intervals):  # loop through split parts, moco and non-moco
    if cnmf_flags[i_segment]:
        for i_frame in range(segment[0] - 1, segment[1]):  # the tuples are indices starting with 1, so subtract 1. Also both inclusive, ccf. range() which is [inclusive, exclusive)
            data_concatenated[i_concat] = images[i_frame]
            i_concat += 1

In [ ]:
#TODO: try to save C-type memmap, then use cell below to open, then compare images2 and images. Should have same dimensions (except T 500 instead of 577)
moco_split_c_memmap = cm.save_memmap([data_concatenated], base_name='mmsplit', order='C',border_to_0=border_to_0, dview=dview) # exclude borders

In [ ]:
moco_split_c_memmap

In [ ]:
# now load the file
Yr2, dims2, T2 = cm.load_memmap(moco_split_c_memmap)
images2 = np.reshape(Yr2.T, [T2] + list(dims2), order='F') 
    #load frames in python format (T x X x Y)

In [ ]:
images.shape

In [ ]:
images2.shape

In [ ]:
# need to save data with non-moco cut out as separate mmap file:
# if does not work, from caiman.mmapping import save_memmap
#FIXME: this does not work, saves 2D data
#concat_fname = cm.save_memmap(filenames = [data_concatenated], base_name="concat", order="C")

### Clean up memory now

In [ ]:
#%% restart cluster to clean up memory
if "dview" in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
cnmf_images = images2  # images2 if split data is used. TODO: add dialog!

### Run CNMF on patches in parallel

In [ ]:
%%capture
#%% RUN CNMF ON PATCHES
# First extract spatial and temporal components on patches and combine them
# for this step deconvolution is turned off (p=0). If you want to have
# deconvolution within each patch change params.patch['p_patch'] to a
# nonzero value
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview, nb_patch = 4)
cnm = cnm.fit(cnmf_images)

In [ ]:
cnmf_images.shape

In [ ]:
#%% plot contours of found components
Cn = cm.local_correlations(cnmf_images.transpose(1,2,0))
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours_nb(img=Cn)

### Inspecting the results
Briefly inspect the results by plotting contours of identified components against correlation image.
The results of the algorithm are stored in the object `cnm.estimates`. More information can be found in the definition of the `estimates` object and in the [wiki](https://github.com/flatironinstitute/CaImAn/wiki/Interpreting-Results).

## Re-run (seeded) CNMF  on the full Field of View  
You can re-run the CNMF algorithm seeded on just the selected components from the previous step. Be careful, because components rejected on the previous step will not be recovered here.

In [ ]:
%%capture
#%% RE-RUN seeded CNMF on accepted patches to refine and perform deconvolution 
cnm2 = cnm.refit(cnmf_images, dview=dview)

## Component Evaluation

The processing in patches creates several spurious components. These are filtered out by evaluating each component using three different criteria:

- the shape of each component must be correlated with the data at the corresponding location within the FOV
- a minimum peak SNR is required over the length of a transient
- each shape passes a CNN based classifier

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

# if performed re-run:
if "cnm2" in locals():
    cnm2.estimates.evaluate_components(cnmf_images, cnm2.params, dview=dview)
else:
    cnm.estimates.evaluate_components(cnmf_images, cnm.params, dview=dview)

Plot contours of selected and rejected components

In [ ]:
#%% PLOT COMPONENTS
if "cnm2" in locals():
    cnm2.estimates.plot_contours_nb(img=Cn, idx=cnm2.estimates.idx_components)
else:
    cnm.estimates.plot_contours_nb(img=Cn, idx=cnm.estimates.idx_components)

View traces of accepted and rejected components. Note that if you get data rate error you can start Jupyter notebooks using:
'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [ ]:
# accepted components
if "cnm2" in locals():
    cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components)
else:
    cnm.estimates.nb_view_components(img=Cn, idx=cnm.estimates.idx_components)

In [ ]:
# rejected components
if "cnm2" in locals():
    if len(cnm2.estimates.idx_components_bad) > 0:
        cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components_bad)
    else:
        print("No components were rejected.")
else:
    if len(cnm.estimates.idx_components_bad) > 0:
        cnm.estimates.nb_view_components(img=Cn, idx=cnm.estimates.idx_components_bad)
    else:
        print("No components were rejected.")

### Extract DF/F values

In [ ]:
#%% Extract DF/F values
#FIXME: "Oops!" printed when cnm2 not in locals (i.e. no refitting was done). Possibly this function never returns.
if "cnm2" in locals():
    cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250)
else:
    cnm.estimates.detrend_df_f(quantileMin=8, frames_window=250)

### Select only high quality components

In [ ]:
if "cnm2" in locals():
    cnm2.estimates.select_components(use_object=True)
else:
    cnm.estimates.select_components(use_object=True)

## Display final results

In [ ]:
if "cnm2" in locals():
    cnm2.estimates.nb_view_components(img=Cn, denoised_color='red')
else:
    cnm.estimates.nb_view_components(img=Cn, denoised_color='red')
print('you may need to change the data rate to generate this one: use jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 before opening jupyter notebook')

## Saving, closing, and creating denoised version
### You can save an hdf5 file with all the fields of the cnmf object. Use load_CNMF() to open the results again

In [ ]:
save_results = True
if save_results:
    if "cnm2" in locals():
        cnm2.save(cnmf_results_save_path)
    else:
        cnm.save(cnmf_results_save_path)
    print(f"saved to\n{cnmf_results_save_path}")

###  Add uuid as attribute to cnmf file.

In [ ]:
with h5py.File(cnmf_results_save_path, 'r+') as hf:
    hf.attrs["uuid"] = session_uuid

### Stop cluster and clean up log files

In [ ]:
#%% STOP CLUSTER and clean up log files
cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)

### Export parameters and metadata as json

In [ ]:
json_dict = opts_dict.copy()
json_dict["original_fnames"] = nd2_fpath
json_dict["rnr_win"] = win
json_dict["amplitude_threshold"] = amplitude_threshold
json_dict["uuid"] = session_uuid

In [ ]:
with open(json_fpath, 'w') as f:
    json.dump(json_dict, f, indent=4)
print(f"Saved parameters to\n{json_fpath}")

### View movie with the results
We can inspect the denoised results by reconstructing the movie and playing alongside the original data and the resulting (amplified) residual movie

In [ ]:
play_movie = False
if play_movie:
    if "cnm2" in locals():
        cnm2.estimates.play_movie(images, q_max=99.9, gain_res=2,
                                          magnification=2,
                                          bpx=border_to_0,
                                          include_bck=False)
    else:
        cnm.estimates.play_movie(images, q_max=99.9, gain_res=2,
                                      magnification=2,
                                      bpx=border_to_0,
                                      include_bck=False)

The denoised movie can also be explicitly constructed using:

In [ ]:
#%% reconstruct denoised movie
if "cnm2" in locals():
    denoised = cm.movie(cnm2.estimates.A.dot(cnm2.estimates.C) + \
                        cnm2.estimates.b.dot(cnm2.estimates.f)).reshape(dims + (-1,), order='F').transpose([2, 0, 1])
else:
    denoised = cm.movie(cnm.estimates.A.dot(cnm.estimates.C) + \
                        cnm.estimates.b.dot(cnm.estimates.f)).reshape(dims + (-1,), order='F').transpose([2, 0, 1])

In [ ]:
save_denoised = False
if save_denoised:
    denoised.save(denoised_optional_fpath)
    print(f"Denoised movie saved to\n\t{denoised_optional_fpath}")

# Save moco parameters

In [ ]:
# motion_correction.py (from caiman) 503-, 524 is the relevant case
Y = cm.load(moco_fnames[0]).astype(np.float32)
ymin = Y.min()
if ymin < 0:
    Y -= Y.min()

xy_grid = [(it[0], it[1]) for it in sliding_window(Y[0], mc.overlaps, mc.strides)]
dims_grid = tuple(np.max(np.stack(xy_grid, axis=1), axis=1) - np.min(
                    np.stack(xy_grid, axis=1), axis=1) + 1)
shifts_x = np.stack([np.reshape(_sh_, dims_grid, order='C').astype(
                    np.float32) for _sh_ in mc.x_shifts_els], axis=0)
shifts_y = np.stack([np.reshape(_sh_, dims_grid, order='C').astype(
                    np.float32) for _sh_ in mc.y_shifts_els], axis=0)

In [ ]:
# TODO: check that MoCo is 0 for the skipped frames!

In [ ]:
moco_params_lis = [
"max_shifts",
"niter_rig",
"splits_rig",
"num_splits_to_process_rig",
"num_splits_to_process_els",
"strides",
"overlaps",
"splits_els",
"upsample_factor_grid",
"max_deviation_rigid",
"shifts_opencv",
"min_mov",
"nonneg_movie",
"gSig_filt",
"use_cuda",
"border_nan",
"pw_rigid",
"var_name_hdf5",
"is3D",
"indices",
"total_template_rig",
"templates_rig",
"fname_tot_rig",
"shifts_rig",
"total_template_els",
"fname_tot_els",
"templates_els",
"x_shifts_els",
"y_shifts_els",
"coord_shifts_els",
"border_to_0",
"mmap_file",  # also fname_mmap_f
]

# min_mov, total_template_rig, total_template_els, border_to_0 have shapes

In [ ]:
list_types = False
if list_types:
    for dset in moco_params_lis:
        data = getattr(mc, dset)
        print(f"{dset}: {type(data)}")
        try:
            print(f"\t{data.shape}")
        except:
            print(f"\tno shape")

In [ ]:
utf8_type = h5py.string_dtype('utf-8', 30)
def append_dataset(h5_file, name, data):
    if (type(data) is tuple and type(data[0]) is slice) \
    or \
    data is None \
    or \
    type(data) is str \
    or \
    (type(data) is list and (data[0] is None or type(data[0]) is str)):  
        # some entries (e.g. indices) are a tuple of slices
        # some entries are of type string, are None, [None, None, ...] or ["..."]
        # convert these types to string (easiest way to preserve information about format)
        #data_arr = np.array(, dtype=utf8_type)
        hf.attrs[name] = data.__str__().encode("utf-8")
    else:
        data_arr = np.array(data)
        dataset = h5_file.create_dataset(name, data_arr.shape, data_arr.dtype)
        if len(data_arr.shape) == 0:
            dataset = data_arr
        else:
            for i in range(data_arr.shape[0]):
                dataset[i] = data_arr[i]

In [ ]:
# TODO: test moco intervals, flags, cnmf flags saved format
with h5py.File(moco_pars_fpath, 'w') as hf:
    print("Adding uuid")
    hf.attrs["uuid"] = session_uuid
    print("Adding MoCo intervals")
    append_dataset(hf, "moco_intervals", moco_intervals)
    print("Adding MoCo flags")
    append_dataset(hf, "moco_flags", moco_flags)
    print("Adding cnmf_intervals")
    append_dataset(hf, "cnmf_intervals", cnmf_intervals)
    print("Adding cnmf_flags")
    append_dataset(hf, "cnmf_flags", cnmf_flags)
    print("Adding begin_end_frames")
    append_dataset(hf, "begin_end_frames", begin_end_frames)
    print("Saving moco params...")
    for dset_name in moco_params_lis:
        print("\t" + dset_name)
        data = getattr(mc, dset_name)
        append_dataset(hf, dset_name, data)
print(f"Saved listed parameters in\n\t{moco_pars_fpath}")

# Opening results (data fields and attributes)

In [ ]:
with h5py.File(moco_pars_fpath, "r") as hf:
    for key in hf.attrs.keys():
        print(f"{key}:\n\t{hf.attrs[key]}")

In [ ]:
with h5py.File(moco_pars_fpath, "r") as hf:
    for key in hf.keys():
        print(f"{key}:\n\t{hf[key]}")